In [3]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

data = pd.read_csv('train.csv')

missing_ratio_threshold = 0.2
missing_values_ratio = data.isnull().sum() / len(data)
columns_to_drop_missing = missing_values_ratio[missing_values_ratio > missing_ratio_threshold].index
data.drop(columns=columns_to_drop_missing, inplace=True)

numeric_columns = data.select_dtypes(include=['number']).columns

selector = VarianceThreshold(threshold=variance_threshold)
selector.fit(data[numeric_columns])
low_variance_cols = data[numeric_columns].columns[~selector.get_support()]
data.drop(columns=low_variance_cols, inplace=True)

numeric_data = data.select_dtypes(include=['number'])

correlation_matrix = numeric_data.corr().abs()

mask = correlation_matrix.applymap(lambda x: x == 1)
correlation_matrix[mask] = 0

high_corr_pairs = (correlation_matrix.stack().sort_values(ascending=False))
high_corr_pairs = high_corr_pairs[high_corr_pairs > 0.8] 
for idx in high_corr_pairs.index:
    feature1, feature2 = idx
    if feature1 in numeric_data.columns and feature2 in numeric_data.columns:
        if correlation_matrix.loc[feature1, :].sum() > correlation_matrix.loc[feature2, :].sum():
            data.drop(columns=[feature2], inplace=True)
        else:
            data.drop(columns=[feature1], inplace=True)

data.to_csv('processed_train.csv', index=False)
